In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [2]:
interpreter = tf.lite.Interpreter(model_path="lite-model_movenet_singlepose_thunder_3.tflite")
interpreter.allocate_tensors()

In [3]:
lowerjoints_list = [13,14,15,16]

In [4]:
def locating_joints(frame, confidence_threshold):
    for i in lowerjoints_list:
        y, x, conf = joints_coordinates[i]
        if conf > confidence_threshold:
            cv2.circle(frame, (int(x), int(y)), 4, (0,255,0), -1)

In [5]:
def knee_distance(frame):
    y1,x1 ,_ = joints_coordinates[13]
    y2, x2,_ = joints_coordinates[14]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame,"Distance:" + str(distance), (0,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [6]:
def left_slope(frame):
  import math
  y1,x1 ,_ = joints_coordinates[15]
  y2, x2,_ = joints_coordinates[13]
  m = (y2-y1)/(x2-x1)
  cv2.putText(frame,"left slope:" + str(m), (50,100), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [7]:
def ankle_distance(frame):
    y1,x1 ,_ = joints_coordinates[15]
    y2, x2,_ = joints_coordinates[16]
    import math
    temp = (x2-x1)**2+(y2-y1)**2
    distance = math.sqrt(temp)
    cv2.putText(frame," Ank:" + str(distance), (150,200), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
    
    

In [8]:
def right_slope(frame):
  import math
  y1,x1 ,_ = joints_coordinates[16]
  y2,x2,_ = joints_coordinates[14]
  m2 = (y2-y1)/(x2-x1)
  cv2.putText(frame,"Right slope:" + str(m2), (100,150), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  

In [9]:
EDGES = {
    (13, 15): 'm',
    (14, 16): 'm',
    (13, 14): 'm',
    (16,15) : 'm'
}

In [10]:
def join_joints(frame, keypoints, edges, confidence_threshold):
    print(keypoints.shape)

    for edge, color in edges.items():
        e1, e2 = edge
        y1, x1, conf1 = joints_coordinates[e1]
        y2, x2, conf2 = joints_coordinates[e2]
        
        
        if (conf1 > confidence_threshold) & (conf2 > confidence_threshold):      
           cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)


In [11]:
def insoles(frame): 
  y1,x1 ,_ = joints_coordinates[13]
  y2,x2,_ = joints_coordinates[14] 
  y3,x3 ,_ = joints_coordinates[15]
  y4,x4,_ = joints_coordinates[16]
  import math 
  dis = (x2-x1)**2+(y2-y1)**2
  ankle_dis = (x4-x3)**2+(y4-y3)**2
  right_m= (y4-y2)/(x4-x2)
  left_m = (y3-y1)/(x3-x1)
  print("knee:", dis)
  print("ankle:", ankle_dis)
  print("left",left_m)
  print("Right",right_m)
#8739.635093408378
  
  #cv2.putText(frame,"insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  ankle = (x4-x3)**2+(y4-y3)**2
  #dis below 88 and ankle below 88, dis >80 and ankle > 80
  if (dis <= 7195.855191894665 and dis > 5608.360385571377 and ankle_dis <= 7195.855191894665 and ankle_dis > 5608.855051436771) or (dis < 7195.855191894665 and dis > 6495.855191894665 and ankle_dis >= 7195.855191894665 and ankle_dis <= 7795.855191894665 ):
     print("running loop 1")
     cv2.putText(frame,"Medium arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis <= 5608.855051436771 and ankle_dis <= 5608.855051436771) or (dis < 5608.855051436771 and ankle_dis > 5608.855051436771 and ankle_dis <= 5908.855051436771):
     #or (dis > 5608.855051436771 and ankle_dis < 7195.855191894665)
     print("running")
     cv2.putText(frame,"Low arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  elif (dis >= 7195.855191894665 and ankle_dis >= 7195.855191894665):
     cv2.putText(frame,"High arch height insole", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
  else:
     cv2.putText(frame,"Other medical condition", (100,250), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
     pass
            
   
    
    

In [12]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _, frame = cap.read()
    
    image = frame.copy()
    image = tf.image.resize_with_pad(np.expand_dims(image, axis=0),256,256)
    input_image = tf.cast(image, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print(output_details)
    
    # predicting joints
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print("hello boy")
    print(keypoints_with_scores)                
    y, x, c = frame.shape
    print("gurs")
    print(np.multiply(keypoints_with_scores, [y,x,1]))
    print("Kaur")
    joints_coordinates = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
    print(joints_coordinates)
    
    locating_joints(frame, 0.4) 
    join_joints(frame, keypoints_with_scores, EDGES, 0.4)
    knee_distance(frame)
    ankle_distance(frame)
    left_slope(frame)
    right_slope(frame)
    insoles(frame)
    cv2.imshow('MoveNet', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[{'name': 'StatefulPartitionedCall:0', 'index': 312, 'shape': array([ 1,  1, 17,  3]), 'shape_signature': array([ 1,  1, 17,  3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
hello boy
[[[[0.53059465 0.5581522  0.61206216]
   [0.47212312 0.595756   0.64228094]
   [0.48195985 0.5060029  0.6209167 ]
   [0.4795657  0.6496393  0.81321895]
   [0.49381882 0.44083393 0.7418963 ]
   [0.69371647 0.7775385  0.56821865]
   [0.70958155 0.30938804 0.80717206]
   [0.77545846 0.91681784 0.05574346]
   [0.84487474 0.25951624 0.01018536]
   [0.76331335 0.94589335 0.04226711]
   [0.77763474 0.3661512  0.02492878]
   [0.8834238  0.97167873 0.01133943]
   [0.80745804 0.27775717 0.01901588]
   [0.71068835 0.7916635  0.0253756 ]
   [0.7338856  0.30317342 0.01301703]
   [0.47268772 0.6024303  0.01283756]
   [0.66356826 0.5654254  0.00252

In [26]:
cap.release()
cv2.destroyAllWindows()